# Jenkins (CI/CD)

## Goals
- Understand what **Jenkins** is and where it fits in CI/CD.
- Know **why** teams use it (tradeoffs, strengths, limitations).
- Learn the core building blocks: **controller/agents**, **pipelines**, **plugins**, **credentials**, and **artifacts**.
- See a realistic **Jenkinsfile pseudo-code** example.


## Prerequisites
- Git + basic branching/PR workflow.
- Basic CI/CD vocabulary: build, test, artifact, deploy.
- Helpful: Docker concepts (images, registries) and basic Linux shell.


## What is Jenkins?

**Jenkins** is an open-source automation server most commonly used to run **CI/CD pipelines**.

At its core, Jenkins:
- Receives an event (commit/push/PR/webhook, schedule, manual trigger).
- Schedules work (a pipeline/job) onto an available worker.
- Runs steps (build/test/package/deploy) and stores results (logs, artifacts, reports).

Jenkins is extremely extensible via a large **plugin ecosystem**.


## What is Jenkins used for (and why teams choose it)?

Common uses:
- **Continuous Integration**: compile/build, run unit/integration tests, run linters, create artifacts.
- **Release pipelines**: versioning, packaging, creating release candidates, publishing artifacts.
- **Continuous Delivery/Deployment**: deploy to environments (dev/stage/prod) with approvals.
- **Automation hub**: scheduled jobs, infra tasks, one-off operational workflows.

Why teams use it:
- **Works anywhere** (self-hosted): on-prem, cloud, air-gapped environments.
- **Flexible pipeline logic** (Pipeline-as-Code via `Jenkinsfile`).
- **Integrates with almost anything** via plugins (SCM, registries, chat, ticketing, cloud).

Why teams avoid it:
- You own the **operations burden** (upgrades, backups, plugin compatibility, scaling, security hardening).


## How Jenkins works (mental model)

### Architecture
- **Controller** (historically called "master"): schedules builds, hosts UI/API, stores config, orchestrates pipelines.
- **Agents** (workers/executors): run the actual steps (build/test/deploy) in isolated environments.
- **Plugins**: add integrations and pipeline steps.

A useful picture:
```
Developer push/PR
      |
      v
   Git provider  --webhook-->  Jenkins controller  --schedules-->  Agent
                                                     |            |
                                                     |            +--> run stages/steps
                                                     |
                                                     +--> store logs/artifacts/reports
```

### Execution flow
1) A trigger starts a job/pipeline.
2) Jenkins loads the pipeline definition (often from a `Jenkinsfile` in the repo).
3) Jenkins allocates an agent (static VM, container, or Kubernetes pod) based on labels/availability.
4) Steps execute in a workspace (checkout code, run commands, publish artifacts, notify).
5) Results are recorded (status, logs, test reports, artifacts).


## Core concepts (minimum you should know)
- **Job**: a configured unit of work (freestyle, pipeline, multibranch, etc.).
- **Pipeline**: a CI/CD workflow described as stages/steps (usually in a `Jenkinsfile`).
- **Stage**: a logical phase (Build, Test, Package, Deploy).
- **Step**: an action inside a stage (shell command, Docker build, publish test results).
- **Agent/Executor**: where work runs; scaling is often done by adding more agents.
- **Workspace**: the agent directory where the repo is checked out and commands run.
- **Artifacts**: build outputs stored for later steps or downloading (packages, binaries, reports).
- **Credentials**: managed secrets (tokens/keys) injected into builds with controlled scope.
- **Shared libraries**: reusable pipeline code to standardize workflows across repos.


## Typical patterns
- **Multibranch pipeline**: Jenkins automatically creates a pipeline per branch/PR based on `Jenkinsfile`.
- **Immutable build environment**: run builds inside Docker/Kubernetes agents for consistency.
- **Promotion**: build once, then promote the same artifact through environments (dev -> stage -> prod).
- **Gates**: require approvals, tests, security scans, or change-management checks before deploy.


## Example: Jenkinsfile (pseudo-code)

Below is illustrative **Declarative Pipeline** style. It is intentionally simplified.

```groovy
// PSEUDO-CODE (Jenkinsfile)
pipeline {
  agent { label 'docker' }  // or 'kubernetes' agent pods

  options {
    timestamps()
    disableConcurrentBuilds()
  }

  environment {
    APP_NAME = 'payments-api'
    IMAGE    = "registry.example.com/${APP_NAME}:${GIT_COMMIT.take(7)}"
  }

  stages {
    stage('Checkout') {
      steps { checkout scm }
    }

    stage('Test') {
      steps {
        sh 'make test'
        junit 'reports/junit.xml'
      }
    }

    stage('Build Artifact') {
      steps {
        sh 'make build'
        archiveArtifacts artifacts: 'dist/**', fingerprint: true
      }
    }

    stage('Build & Push Image') {
      when { branch 'main' }
      steps {
        withCredentials([usernamePassword(credentialsId: 'registry-creds', usernameVariable: 'USER', passwordVariable: 'PASS')]) {
          sh 'docker login -u $USER -p $PASS registry.example.com'
          sh 'docker build -t $IMAGE .'
          sh 'docker push $IMAGE'
        }
      }
    }

    stage('Deploy to Staging') {
      when { branch 'main' }
      steps {
        sh 'deploy --env staging --image $IMAGE'  // replace with your deploy mechanism
      }
    }

    stage('Deploy to Prod (Approval)') {
      when { branch 'main' }
      steps {
        input message: 'Deploy to production?', ok: 'Deploy'
        sh 'deploy --env prod --image $IMAGE'
      }
    }
  }

  post {
    always { cleanWs() }
    failure { echo 'Notify: build failed' }
  }
}
```


## What Jenkins is capable of
- Large-scale CI with distributed agents (VMs, containers, Kubernetes pods).
- Strong support for **enterprise constraints** (private networks, custom tooling, air-gapped).
- Highly customizable pipelines and integrations via plugins + shared libraries.

Good fit when:
- You need maximum flexibility and must run in your own infrastructure.

Less ideal when:
- You want a low-maintenance managed CI/CD with fewer moving parts.


## Pitfalls & quick tips
- Treat the controller as **critical infrastructure**: backups, HA, and controlled upgrades.
- Minimize plugin sprawl; pin versions and regularly review plugin CVEs/updates.
- Prefer **ephemeral agents** (containers/pods) to avoid drift and cross-build contamination.
- Use least-privilege credentials; avoid long-lived static cloud keys when possible.
- Keep pipelines deterministic: same inputs -> same outputs.

## Exercises
- Convert an existing shell script build into a multistage Jenkins pipeline.
- Add test report publishing (JUnit) and artifact archiving.
- Implement a manual approval gate for production deploys.

## References
- Jenkins Docs: https://www.jenkins.io/doc/
- Pipeline Syntax: https://www.jenkins.io/doc/book/pipeline/syntax/
- Jenkinsfile: https://www.jenkins.io/doc/book/pipeline/jenkinsfile/
